In [11]:
import os
import re
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

In [2]:
# --- 설정: 경로 지정 ---
wav_paths = [
    "unwelcomeSchool/vocals.wav",
    "unwelcomeSchool/bass.wav",
    "unwelcomeSchool/drums.wav",
    "unwelcomeSchool/other.wav"
]


osu_input = "UnwelcomeSchool_Hard.txt"
osu_output = "UnwelcomeSchool_Hard.osu"
output_dir = os.path.join(os.path.dirname(osu_output), "splited_notes")
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def smooth_assignments(assignments, window=2):
    """
    assignments: [int] 노트별 dominant wav 인덱스 리스트 (예: [3,3,3,2,3,3])
    window: 좌우 몇 개 노트를 참고할지
    
    리턴값: 스무딩된 assignments 리스트
    """
    n = len(assignments)
    smoothed = assignments.copy()
    
    for i in range(n):
        # 좌우 window 범위 내 인덱스들 수집
        start = max(0, i - window)
        end = min(n, i + window + 1)
        window_vals = assignments[start:end]
        
        # 가장 많은 값으로 대체 (다수결)
        from collections import Counter
        c = Counter(window_vals)
        most_common = c.most_common(1)[0][0]
        
        smoothed[i] = most_common
    
    return smoothed

In [3]:
# 증폭할 트랙 인덱스와 증폭 비율 (예: other(3)만 증폭)
amplify_indices = [3]
amplify_factor = 1.5

# 전역 클립 길이 (ms) 및 페이드 인/아웃 길이 (ms)
default_clip_ms = 333
fade_ms = 10

# --- 1) WAV 파일 로드 및 증폭 적용 ---
waves = []
sr = None
for idx, p in enumerate(wav_paths):
    y, sr_tmp = librosa.load(p, sr=None)
    if sr is None:
        sr = sr_tmp
    elif sr != sr_tmp:
        raise ValueError("WAV 파일 간 샘플링 레이트 불일치")
    if idx in amplify_indices:
        y = np.clip(y * amplify_factor, -1.0, 1.0)
    waves.append(y)

In [4]:
# --- 2) osu 파일 파싱 ---
with open(osu_input, 'r', encoding='utf-8') as f:
    osu_text = f.read()

In [5]:
# HitObjects 타이밍 리스트 구성
hit_match = re.search(r"\[HitObjects\](.*)$", osu_text, re.S)
if not hit_match:
    raise ValueError("[HitObjects] 섹션을 찾을 수 없습니다.")
hit_lines = [l for l in hit_match.group(1).strip().splitlines() if l.strip()]
times_ms = [int(line.split(',')[2]) for line in hit_lines]

# Events 영역 위치 캡처
events_match = re.search(r"\[Events\](.*?)(?=\n\[|$)", osu_text, re.S)

In [6]:
# --- 3) 페이드 윈도우 생성 ---
fade_len = int(sr * fade_ms / 1000)
ramp = np.linspace(0, 1, fade_len)

In [7]:
# --- 4) 클립 추출 및 노트 적용 ---
silent_tracks = [w.copy() for w in waves]
new_events, new_hit_lines = [], []
counter = 1


In [12]:
for i, line in tqdm(enumerate(hit_lines), total=len(hit_lines), desc="Generating clips"):
    parts = line.split(',')
    time_ms = times_ms[i]
    # 다음 노트 간격에 맞춰 클립 길이 제한
    if i < len(times_ms) - 1:
        next_ms = times_ms[i+1]
        clip_ms = min(default_clip_ms, next_ms - time_ms)
    else:
        clip_ms = default_clip_ms
    clip_len = int(sr * clip_ms / 1000)

    tail = parts[-1]
    effects, filename = (tail.rsplit(':', 1) if ':' in tail else (tail, ''))

    start = int(sr * time_ms / 1000)
    end = min(start + clip_len, len(waves[0]))

    # 에너지 비교 및 dominant 트랙 선택
    energies = [np.sum(w[start:end]**2) for w in waves]
    idx = int(np.argmax(energies))

    # 클립 추출
    clip = waves[idx][start:end].copy()
    # 페이드 인/아웃 적용
    if len(clip) > 2 * fade_len:
        clip[:fade_len] *= ramp
        clip[-fade_len:] *= ramp[::-1]
    else:
        clip *= np.linspace(0, 1, len(clip))

    # 무음 처리 적용 (역페이드)
    seg = silent_tracks[idx][start:end]
    if len(seg) > 2 * fade_len:
        seg[:fade_len] *= (1 - ramp)
        seg[-fade_len:] *= (1 - ramp[::-1])
    silent_tracks[idx][start:end] = 0

    # 클립 저장
    clip_name = f"note_{idx+1}_{counter}.wav"
    sf.write(os.path.join(output_dir, clip_name), clip, sr)

    # HitObjects 및 Events 업데이트
    parts[-1] = f"{effects}:{clip_name}"
    new_hit_lines.append(','.join(parts))
    new_events.append(f"Sample,{time_ms},0,\"{clip_name}\",50")
    counter += 1

Generating clips: 100%|███████████████████████████████████████████████████████████| 1338/1338 [00:04<00:00, 325.22it/s]


In [9]:
# --- 5) accompaniment 생성 ---
accompaniment = np.zeros_like(waves[0])
for tr in silent_tracks:
    accompaniment += tr
sf.write(os.path.join(output_dir, "accompaniment.wav"), accompaniment, sr)


In [10]:
# --- 6) 수정된 osu 파일 작성 ---
header = osu_text[:events_match.start()]
between = osu_text[events_match.end():hit_match.start()]
footer = osu_text[hit_match.end():]
with open(osu_output, 'w', encoding='utf-8') as f:
    f.write(header)
    f.write("[Events]\n")
    for ev in new_events:
        f.write(ev + "\n")
    f.write("\n" + between)
    f.write("[HitObjects]\n")
    for hl in new_hit_lines:
        f.write(hl + "\n")
    f.write(footer)

print(f"완료: 겹침 방지, 페이드 처리 및 accompaniment 생성 완료. 파일 위치: {output_dir}")


완료: 겹침 방지, 페이드 처리 및 accompaniment 생성 완료. 파일 위치: splited_notes
